In [1]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [2]:
require("DefinitionPara.jl")

 in depwarn at deprecated.jl:73
 [inlined code] from deprecated.jl:694
 in require at no file:0
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at C:\Users\Minsu Chang\.julia\v0.4\IJulia\src\execute_request.jl:177
 in eventloop at C:\Users\Minsu Chang\.julia\v0.4\IJulia\src\IJulia.jl:141
 in anonymous at task.jl:447
while loading In[2], in expression starting on line 1


In [3]:
function main_paral()

aalpha = 1/3     # Elasticity of output w.r.t. capital
bbeta  = 0.95    # Discount factor

# Productivity values
vProductivity = [0.9792 0.9896 1.0000 1.0106 1.0212]

# Transition matrix
mTransition   =     [0.9727 0.0273 0.0000 0.0000 0.0000;
                     0.0041 0.9806 0.0153 0.0000 0.0000;
                     0.0000 0.0082 0.9837 0.0082 0.0000;
                     0.0000 0.0000 0.0153 0.9806 0.0041;
                     0.0000 0.0000 0.0000 0.0273 0.9727]

    # 2. Steady State

    capitalSteadyState = (aalpha*bbeta)^(1/(1-aalpha))
  
    # We generate the grid of capital
vGridCapital = collect(0.5*capitalSteadyState:0.0001:1.5*capitalSteadyState)

    nGridCapital = length(vGridCapital)
    nGridProductivity = length(vProductivity)

    # 3. Required matrices and vectors

mOutput           = zeros(nGridCapital,nGridProductivity)
mValueFunction    = zeros(nGridCapital,nGridProductivity)
mValueFunctionNew = SharedArray(Float64, (nGridCapital,nGridProductivity))
mPolicyFunction   = SharedArray(Float64, (nGridCapital,nGridProductivity))
expectedValueFunction = zeros(nGridCapital,nGridProductivity)

    # 4. We pre-build output for each point in the grid

mOutput = (vGridCapital.^aalpha)*vProductivity

    # 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.0000001
    iteration = 0

pparams = [aalpha, bbeta, nGridCapital, nGridProductivity]   

#while(maxDifference > tolerance)
    expectedValueFunction = mValueFunction*mTransition';  
    
    final_shared(mValueFunctionNew, mPolicyFunction, pparams, vGridCapital, mOutput, expectedValueFunction)
    
    maxDifference = maximum(abs(mValueFunctionNew - mValueFunction))
    mValueFunction = sdata(mValueFunctionNew)
    mValueFunctionNew =  SharedArray(Float64, (nGridCapital,nGridProductivity))
    
    iteration = iteration+1
        if mod(iteration,10)==0 || iteration == 1
            println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
        end
    
#end

    return mValueFunction, mPolicyFunction
end


main_paral (generic function with 1 method)

In [23]:
@time main_paral();

  0.000714 seconds (543 allocations: 51.578 KB)
  0.000640 seconds (543 allocations: 51.578 KB)
 Iteration = 1 Sup Diff = 0.05274159340733662
  0.004515 seconds (4.06 k allocations: 816.734 KB)


In [5]:
function main()

    aalpha = 1/3     # Elasticity of output w.r.t. capital
    bbeta  = 0.95    # Discount factor

    # Productivity values
    vProductivity = [0.9792 0.9896 1.0000 1.0106 1.0212]

    # Transition matrix
    mTransition   = [0.9727 0.0273 0.0000 0.0000 0.0000;
                     0.0041 0.9806 0.0153 0.0000 0.0000;
                     0.0000 0.0082 0.9837 0.0082 0.0000;
                     0.0000 0.0000 0.0153 0.9806 0.0041;
                     0.0000 0.0000 0.0000 0.0273 0.9727]

    # 2. Steady State

    capitalSteadyState = (aalpha*bbeta)^(1/(1-aalpha))

    # We generate the grid of capital
    vGridCapital = collect(0.5*capitalSteadyState:0.0001:1.5*capitalSteadyState)

    nGridCapital = length(vGridCapital)
    nGridProductivity = length(vProductivity)

    # 3. Required matrices and vectors

    mOutput           = zeros(nGridCapital,nGridProductivity)
    mValueFunction    = zeros(nGridCapital,nGridProductivity)
    mValueFunctionNew = zeros(nGridCapital,nGridProductivity)
    mPolicyFunction   = zeros(nGridCapital,nGridProductivity)
    expectedValueFunction = zeros(nGridCapital,nGridProductivity)

    # 4. We pre-build output for each point in the grid

    mOutput = (vGridCapital.^aalpha)*vProductivity;

    # 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.0000001
    iteration = 0

   # while(maxDifference > tolerance)
        expectedValueFunction = mValueFunction*mTransition';

        for nProductivity = 1:nGridProductivity
        
            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 1
        
            for nCapital = 1:nGridCapital
        
                valueHighSoFar = -1000.0
                capitalChoice  = vGridCapital[1]
            
                for nCapitalNextPeriod = gridCapitalNextPeriod:nGridCapital

                    consumption = mOutput[nCapital,nProductivity]-vGridCapital[nCapitalNextPeriod]
                    valueProvisional = (1-bbeta)*log(consumption)+bbeta*expectedValueFunction[nCapitalNextPeriod,nProductivity]
               
                    if (valueProvisional>valueHighSoFar)
                	valueHighSoFar = valueProvisional
                	capitalChoice = vGridCapital[nCapitalNextPeriod]
                	gridCapitalNextPeriod = nCapitalNextPeriod
                    else
                	break # We break when we have achieved the max
                    end
                                 
                end
            
                mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
                mPolicyFunction[nCapital,nProductivity] = capitalChoice
          
            end
        
        end

        maxDifference  = maximum(abs(mValueFunctionNew-mValueFunction))
        mValueFunction    = mValueFunctionNew
        mValueFunctionNew = zeros(nGridCapital,nGridProductivity)

        iteration = iteration+1
        if mod(iteration,10)==0 || iteration == 1
            println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
        end
           
   # end

end

main (generic function with 1 method)

In [27]:
@time main()

 Iteration = 1 Sup Diff = 0.05274159340733662
  0.001041 seconds (349 allocations: 740.500 KB)
